In [2]:


import tensorflow as tf
# Initialize the TPU
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# # Create a TPU strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

strategy = tf.distribute.MirroredStrategy()



2023-12-06 23:50:27.756435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 23:50:27.756491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 23:50:27.757418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 23:50:27.763722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 23:50:28.675420: W tensorflow/compiler/tf2

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [3]:
import numpy as np

import os
import cv2
import glob
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D  # Import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [35]:
# Define the list of labels
labels = ["broadleaf", "grass", "soil", "soybean"]

# Load the dataset and preprocess the images
X = []
Y = []
image_size = (224, 224)  # Adjust to match VGG16 input size

for label in labels:
    dataset_path = "/home/samuel/NYU/DBS/proj/lake/cv/dataset/" + label + "/"
    print(dataset_path)
    file_paths = glob.glob(dataset_path + "*")
    counter = 0
    for file_path in file_paths:
        image = cv2.imread(file_path)
        img = cv2.resize(image, image_size)
        img_flattened = img.flatten()
        counter = counter + 1
        if counter%50 == 0:
            X.append(img_flattened)
            Y.append(label)

# Convert labels to numerical values
label_to_index = {label: index for index, label in enumerate(labels)}
Y = [label_to_index[label] for label in Y]

# Convert lists to NumPy arrays
X = np.array(X)
Y = np.array(Y)

/home/samuel/NYU/DBS/proj/lake/cv/dataset/broadleaf/
/home/samuel/NYU/DBS/proj/lake/cv/dataset/grass/
/home/samuel/NYU/DBS/proj/lake/cv/dataset/soil/
/home/samuel/NYU/DBS/proj/lake/cv/dataset/soybean/


In [4]:
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=101)

# Preprocess the images for VGG16
X_train = tf.keras.applications.vgg16.preprocess_input(X_train)
X_test = tf.keras.applications.vgg16.preprocess_input(X_test)

In [5]:

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(labels), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history =model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", test_accuracy)

Epoch 1/10
7/7 [==============================] - 297s 43s/step - loss: 63.3998 - accuracy: 0.3396 - val_loss: 3.2452 - val_accuracy: 0.2174
Epoch 2/10
7/7 [==============================] - 320s 47s/step - loss: 1.7417 - accuracy: 0.4057 - val_loss: 1.3380 - val_accuracy: 0.5109
Epoch 3/10
7/7 [==============================] - 343s 47s/step - loss: 1.3595 - accuracy: 0.4009 - val_loss: 1.3297 - val_accuracy: 0.1522
Epoch 4/10
7/7 [==============================] - 323s 48s/step - loss: 1.2546 - accuracy: 0.4481 - val_loss: 1.2902 - val_accuracy: 0.5326
Epoch 5/10
7/7 [==============================] - 279s 40s/step - loss: 1.2577 - accuracy: 0.4623 - val_loss: 1.2685 - val_accuracy: 0.5326
Epoch 6/10
7/7 [==============================] - 328s 48s/step - loss: 1.2523 - accuracy: 0.4623 - val_loss: 1.2805 - val_accuracy: 0.5326
Epoch 7/10
7/7 [==============================] - 343s 48s/step - loss: 1.2592 - accuracy: 0.4623 - val_loss: 1.2868 - val_accuracy: 0.5326
Epoch 8/10
7/7 [===

In [9]:
# Save the model architecture as JSON
model_json = model.to_json()

# Specify the file path where you want to save the JSON file
json_file_path = './model.json'
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")

In [ ]:
model.to_json()